In [5]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Custom dataset for FFHQ
class FFHQDataset(Dataset):
    def __init__(self, hr_dir, lr_dir, transform=None):
        self.hr_dir = hr_dir  # High-Resolution images
        self.lr_dir = lr_dir  # Low-Resolution images
        self.hr_images = os.listdir(hr_dir)
        self.lr_images = os.listdir(lr_dir)
        self.transform = transform

    def __len__(self):
        return len(self.hr_images)

    def __getitem__(self, idx):
        hr_path = os.path.join(self.hr_dir, self.hr_images[idx])
        lr_path = os.path.join(self.lr_dir, self.lr_images[idx])
        
        hr_image = Image.open(hr_path).convert('RGB')
        lr_image = Image.open(lr_path).convert('RGB')
        
        if self.transform:
            hr_image = self.transform(hr_image)
            lr_image = self.transform(lr_image)
        
        return lr_image, hr_image

ModuleNotFoundError: No module named 'torch'

In [4]:
import os

# Path to the dataset directory
dataset_path = "/kaggle/input/flickrfaceshq-dataset-ffhq"  # Adjust based on where the dataset is located

# List all files and subdirectories inside the dataset
print(dataset_path)
print(len(dataset_path))

/kaggle/input/flickrfaceshq-dataset-ffhq
40


In [ ]:
import torch
# from byol_pytorch import BYOL
from torchvision import models
import os
import numpy as np
import torch
from torchvision.transforms import transforms
from PIL import Image

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_names = os.listdir(root_dir)
        self.file_names.sort()

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        file_path = os.path.join(self.root_dir, self.file_names[idx])
        image = Image.open(file_path)
        if self.transform:
            image = self.tran

In [ ]:
dataset = CustomDataset('/kaggle/input/flickrfaceshq-dataset-ffhq', transform=transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
print(len(dataloader))

In [ ]:
import os

# Path to the dataset directory
dataset_path = "/kaggle/input/flickrfaceshq-dataset-ffhq"  # Update this with the correct path

# List of valid image extensions (you can add more if needed)
valid_image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')

# List to store paths of image files
image_files = []

# Walk through all directories and subdirectories
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        # Check if the file is an image based on the extension
        if file.lower().endswith(valid_image_extensions):
            image_files.append(os.path.join(root, file))

# Print total number of image files found
print(f"Total number of image files: {len(image_files)}")


In [ ]:
from PIL import Image
import os

# Path to the high-resolution images (HR images)
hr_dir = "/kaggle/input/flickrfaceshq-dataset-ffhq"  # Adjust this path based on the actual structure

# Path to save low-resolution images (LR images)
lr_dir = "/kaggle/working/lr-images"  # Adjust as needed (ensure it's writable)

# Create LR directory if it doesn't exista
if not os.path.exists(lr_dir):
    os.makedirs(lr_dir)

resize_transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to 256x256
    transforms.ToTensor(),          # Convert to tensor
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Downscale HR images to generate LR images (e.g., 256x256)
count = 0  # Counter for processed images
for img_name in os.listdir(hr_dir):
    img_path = os.path.join(hr_dir, img_name)
    
    if os.path.isfile(img_path):  # Check if it's a valid file
        try:
            # Open image and convert to RGB
            img = Image.open(img_path).convert('RGB')
            
            # Apply transformation
            img_tensor = resize_transform(img).to(device)  # Move image to GPU
            
            # Convert back to PIL image
            img_resized = transforms.ToPILImage()(img_tensor.cpu())  # Move back to CPU
            
            # Save resized image
            img_resized.save(os.path.join(lr_dir, img_name))
            
            count += 1
        except Exception as e:
            print(f"Error processing {img_name}: {e}")

print(f"Done! Converted {count} images to low resolution and saved to {lr_dir}.")
Using device: cuda

In [ ]:
lr_dir = "/kaggle/input/lr-images-root"

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [ ]:
class FFHQDataset(Dataset):
    def __init__(self, hr_dir, lr_dir, transform=None):
        self.hr_dir = hr_dir
        self.lr_dir = lr_dir
        self.hr_images = sorted(os.listdir(hr_dir))
        self.lr_images = sorted(os.listdir(lr_dir))
        self.transform = transform

    def __len__(self):
        return min(len(self.hr_images), len(self.lr_images))  # Ensure lengths match

    def __getitem__(self, idx):
        hr_path = os.path.join(self.hr_dir, self.hr_images[idx])
        lr_path = os.path.join(self.lr_dir, self.lr_images[idx])
        
        hr_image = Image.open(hr_path).convert('RGB')
        lr_image = Image.open(lr_path).convert('RGB')

        # Resize to expected dimensions if necessary
        hr_image = hr_image.resize((512, 512), Image.Resampling.LANCZOS)
        lr_image = lr_image.resize((256, 256), Image.Resampling.LANCZOS)
        
        if self.transform:
            hr_image = self.transform(hr_image)
            lr_image = self.transform(lr_image)
        
        return lr_image, hr_image

In [ ]:
print(f"Number of HR images: {len(dataset.hr_images)}")
print(f"Number of LR images: {len(dataset.lr_images)}")

In [ ]:
for idx in range(len(dataset)):
    lr_image, hr_image = dataset[idx]
    print(f"LR Image {idx}: {lr_image.size}, HR Image {idx}: {hr_image.size}")
    break

In [ ]:
import torch.nn as nn

# Define a simple ESRGAN-like model for super-resolution
class ESRGANGenerator(nn.Module):
    def __init__(self):
        super(ESRGANGenerator, self).__init__()
        self.upsample = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3 * 4, kernel_size=3, padding=1),  # For 2x upscaling
            nn.PixelShuffle(2)  # 2x upscaling
        )

    def forward(self, x):
        return self.upsample(x)

In [ ]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize between -1 and 1
    ])
dataset = FFHQDataset(hr_dir=hr_dir, lr_dir=lr_dir, transform=transform)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
def train_model(model, train_loader, device, num_epochs=1, lr=0.0001):
    criterion = nn.MSELoss()  # Loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Optimizer
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for lr_images, hr_images in train_loader:
            lr_images = lr_images.to(device)
            hr_images = hr_images.to(device)
            
            # Forward pass
            outputs = model(lr_images)
            
            # Compute loss
            loss = criterion(outputs, hr_images)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
# Step 4: Train the model
device = torch.device("cud

In [ ]:
# Step 4: Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ESRGANGenerator().to(device)
train_model(model, train_loader, device, num_epochs=1)